In [1]:
from __future__ import division
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.special import gamma as gam
from scipy.misc import factorial as fac

In [3]:
%load_ext Cython

In [29]:
%%file install.sh

# Make sure we have cythongsl
wget ftp://ftp.gnu.org/gnu/gsl/gsl-latest.tar.gz
tar -xzf gsl-latest.tar.gz
cd gsl-1.16
# The next three lines seem to have an issue
pwd
./configure --prefix=/usr/local
make
make install

Writing install.sh


In [30]:
! chmod +x install.sh
! ./install.sh

--2015-04-15 20:06:43--  ftp://ftp.gnu.org/gnu/gsl/gsl-latest.tar.gz
           => 'gsl-latest.tar.gz'
Resolving ftp.gnu.org... 208.118.235.20, 2001:4830:134:3::b
Connecting to ftp.gnu.org|208.118.235.20|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /gnu/gsl ... done.
==> SIZE gsl-latest.tar.gz ... 3534080
==> PASV ... done.    ==> RETR gsl-latest.tar.gz ... done.
Length: 3534080 (3.4M) (unauthoritative)

gsl-latest.tar.gz   100%[=====================>]   3.37M  2.25MB/s   in 1.5s   

2015-04-15 20:06:45 (2.25 MB/s) - 'gsl-latest.tar.gz' saved [3534080]

/Users/cliburn/git/sta-633/Debug/gsl-1.16
checking for a BSD-compatible install... /usr/bin/install -c
checking whether build environment is sane... yes
checking for a thread-safe mkdir -p... ./install-sh -c -d
checking for gawk... no
checking for mawk... no
checking for nawk... no
checking for awk... awk
checking whether make sets $(MAKE)... yes
check

In [28]:
# # Make sure we have cythongsl
# ! wget ftp://ftp.gnu.org/gnu/gsl/gsl-latest.tar.gz
# ! tar -xzf gsl-latest.tar.gz
# ! cd gsl-1.16
# # The next three lines seem to have an issue
# ! pwd
# ! ./configure --prefix=/usr/local
# ! make
# ! make install

In [6]:
# This seems to indicate that I have cython gsl installed
! pip install cythongsl 

Cleaning up...


In [31]:
%%cython -a -lgsl
import cython
import numpy as np
cimport numpy as np
from cython_gsl cimport gsl_rng_mt19937, gsl_rng, gsl_rng_alloc, gsl_rng_uniform

cdef gsl_rng *r = gsl_rng_alloc(gsl_rng_mt19937)

@cython.cdivision
@cython.boundscheck(False)
@cython.wraparound(False)
def pi_cython(int n):
    cdef int[:] s = np.zeros(n, dtype=np.int32)
    cdef int i = 0
    cdef double x, y
    for i in range(n):
        x = gsl_rng_uniform(r)*2 - 1
        y = gsl_rng_uniform(r)*2 - 1
        s[i] = x**2 + y**2 < 1
    cdef int hits = 0
    for i in range(n):
        hits += s[i]
    return 4.0*hits/n